<a href="https://colab.research.google.com/github/nike-2001/Toxicity-Detection-In-Social-Media/blob/main/Detoxifying_Toxic_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install essential libraries for data manipulation, machine learning, and NLP
pip install pandas numpy tensorflow transformers torch scikit-learn nltk


In [ ]:
# import necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import re
from torch.utils.data import DataLoader
from nltk.translate.bleu_score import sentence_bleu


In [ ]:
# Check if a GPU is available, otherwise fallback to the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print the device being used (e.g., 'cuda' for GPU or 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [ ]:
# Import the files module from Google Colab to handle file uploads
from google.colab import files

# Upload a file from your local machine (e.g., 'train.csv.zip')
files.upload()


Saving train.csv.zip to train.csv.zip


In [ ]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [ ]:
# Read the dataset from the CSV file (replace 'train.csv' with the correct file path)
data = pd.read_csv('train.csv')

# Select only the relevant columns: 'comment_text' and 'toxic'
data = data[['comment_text', 'toxic']]

# Display the first few rows of the dataset
print(data.head())


                                        comment_text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0


In [ ]:
import re  # Import the regular expressions library for text processing

# Define a function to clean the text
def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+", "", text)
    # Remove non-alphanumeric characters (except spaces)
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)
    # Replace multiple spaces with a single space and strip leading/trailing spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply the clean_text function to the 'comment_text' column and store the results in a new column
data['cleaned_text'] = data['comment_text'].apply(clean_text)

# Display the first few rows of the cleaned text
print(data['cleaned_text'].head())


0    Explanation Why the edits made under my userna...
1    Daww He matches this background colour Im seem...
2    Hey man Im really not trying to edit war Its j...
3    More I cant make any real suggestions on impro...
4    You sir are my hero Any chance you remember wh...
Name: cleaned_text, dtype: object


In [ ]:
# Split the dataset into training and validation sets
# 'train_test_split' divides the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['cleaned_text'],  # Input texts (cleaned)
    data['toxic'],         # Corresponding labels (toxic or not)
    test_size=0.2,         # 20% of the data will be used for validation
    random_state=42        # Ensures reproducibility of the split
)


In [ ]:
# Import the T5 tokenizer and model for text-to-text generation
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained T5 tokenizer (small version)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Load the pre-trained T5 model for conditional generation (small version)
model = T5ForConditionalGeneration.from_pretrained("t5-small")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Define a function to encode texts using the tokenizer
def encode_texts(texts, max_length=512):
    return tokenizer(
        texts.tolist(),         # Convert the input texts to a list
        max_length=max_length,  # Set the maximum token length
        padding="max_length",   # Pad all sequences to the maximum length
        truncation=True,        # Truncate sequences longer than the maximum length
        return_tensors="pt"     # Return PyTorch tensors
    )

# Encode the training texts into tokenized representations
train_encodings = encode_texts(train_texts)

# Encode the validation texts into tokenized representations
val_encodings = encode_texts(val_texts)


In [ ]:
# Detoxify text in batches
def batch_detoxify_texts(texts, batch_size=16):
    dataloader = DataLoader(texts, batch_size=batch_size)
    detoxified_texts = []

    # Move model to the same device as the input data
    model.to(device)  # This line is added to move the model to the device

    for batch in dataloader:
        inputs = tokenizer(list(batch), return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = model.generate(inputs['input_ids'])
        detoxified_texts.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])

    return detoxified_texts

# Perform detoxification
data['detoxified_text'] = batch_detoxify_texts(data['cleaned_text'].tolist())
print(data[['cleaned_text', 'detoxified_text']].head())


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


                                        cleaned_text  \
0  Explanation Why the edits made under my userna...   
1  Daww He matches this background colour Im seem...   
2  Hey man Im really not trying to edit war Its j...   
3  More I cant make any real suggestions on impro...   
4  You sir are my hero Any chance you remember wh...   

                                     detoxified_text  
0  Explanation Why the edits made under my userna...  
1  Daww He matches this background colour Im seem...  
2  Im really not trying to edit war Im really not...  
3  if you have any preferences for formatting sty...  
4  You sir are my hero Any chance you remember wh...  


In [ ]:
# Import the BLEU score function from NLTK for evaluation
from nltk.translate.bleu_score import sentence_bleu

# Define a function to compute the BLEU score between the original and transformed text
def evaluate_bleu(original, transformed):
    # Calculate BLEU score based on word-level comparison
    return sentence_bleu([original.split()], transformed.split())

# Apply the BLEU evaluation function to each row of the dataset
data['bleu_score'] = data.apply(
    lambda x: evaluate_bleu(x['cleaned_text'], x['detoxified_text']), axis=1
)

# Display the first few rows with original text, transformed text, and BLEU scores
print(data[['cleaned_text', 'detoxified_text', 'bleu_score']].head())


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

                                        cleaned_text  \
0  Explanation Why the edits made under my userna...   
1  Daww He matches this background colour Im seem...   
2  Hey man Im really not trying to edit war Its j...   
3  More I cant make any real suggestions on impro...   
4  You sir are my hero Any chance you remember wh...   

                                     detoxified_text  bleu_score  
0  Explanation Why the edits made under my userna...    0.090752  
1  Daww He matches this background colour Im seem...    0.939413  
2  Im really not trying to edit war Im really not...    0.154378  
3  if you have any preferences for formatting sty...    0.004464  
4  You sir are my hero Any chance you remember wh...    1.000000  


In [ ]:
# Import the BLEU score function from NLTK
from nltk.translate.bleu_score import sentence_bleu

# Function to calculate the BLEU score between original and transformed text
def evaluate_bleu(original, transformed):
    return sentence_bleu([original.split()], transformed.split())

# Apply the BLEU score function to each row in the dataset
data['bleu_score'] = data.apply(
    lambda x: evaluate_bleu(x['cleaned_text'], x['detoxified_text']), axis=1
)

# Print the first few rows with original text, transformed text, and BLEU scores
print(data[['cleaned_text', 'detoxified_text', 'bleu_score']].head())


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

                                        cleaned_text  \
0  Explanation Why the edits made under my userna...   
1  Daww He matches this background colour Im seem...   
2  Hey man Im really not trying to edit war Its j...   
3  More I cant make any real suggestions on impro...   
4  You sir are my hero Any chance you remember wh...   

                                     detoxified_text  bleu_score  
0  Explanation Why the edits made under my userna...    0.090752  
1  Daww He matches this background colour Im seem...    0.939413  
2  Im really not trying to edit war Im really not...    0.154378  
3  if you have any preferences for formatting sty...    0.004464  
4  You sir are my hero Any chance you remember wh...    1.000000  


In [ ]:
data.to_csv("detoxified_comments.csv", index=False)


In [ ]:
pip install openai==0.27.8


In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "API_KEY"

def detoxify_with_gpt(prompt: str) -> str:
    """
    Detoxifies a given text using the OpenAI GPT model.

    Args:
        prompt (str): Input text containing potentially toxic language.

    Returns:
        str: Detoxified version of the input text.
    """
    # Define a system prompt for detoxification
    system_prompt = (
        "You are an assistant specializing in detoxifying text. "
        "When provided with a toxic comment, rephrase it to reduce harmful or offensive language "
        "while maintaining the original intent."
    )

    try:
        # Call the GPT model
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # You can use "gpt-3.5-turbo" if you want faster and cheaper responses
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            max_tokens=100,
            temperature=0.7
        )

        # Extract the response text
        detoxified_text = response['choices'][0]['message']['content']
        return detoxified_text.strip()
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Interactive input for detoxification
if __name__ == "__main__":
    user_prompt = input("Enter a toxic text to detoxify: ")
    detoxified_output = detoxify_with_gpt(user_prompt)
    print("\nDetoxified Output:")
    print(detoxified_output)


Enter a toxic text to detoxify: you idiot

Detoxified Output:
You made a mistake.
